In [2]:
from dotenv import load_dotenv

load_dotenv()
import os

import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec



In [3]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

pc.create_index(
    name='rag', dimension= 768, metric= 'cosine', spec=ServerlessSpec(cloud="aws", region='us-east-1')
)

In [12]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Physics',
  'stars': 5,
  'review': "Dr. Johnson's lectures are incredibly engaging and she explains complex concepts with great clarity. Her passion for physics is contagious!"},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 4,
  'review': 'Prof. Chen is knowledgeable and provides practical coding examples. Sometimes his pace is a bit fast, but overall a great instructor.'},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Dr. Williams covers the material well, but her assignments can be vague. Office hours are helpful for clarification.'},
 {'professor': 'Prof. David Martinez',
  'subject': 'Literature',
  'stars': 5,
  'review': 'Prof. Martinez brings literature to life! His enthusiasm and deep understanding of the texts make every class a joy to attend.'},
 {'professor': 'Dr. Lisa Thompson',
  'subject': 'Biology',
  'stars': 2,
  'review': 'Dr. Thompson se

In [20]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
processed_data = []


for review in data["reviews"]:
    print(review)
    response = genai.embed_content(
        model="models/text-embedding-004",
        text=review['review']
    )

    embedding = response.data[0].embedding
    processed_data.append({
        "professor": review["professor"],
        "subject": review["subject"],
        "stars": review["stars"],
        "embedding": embedding
    })

{'professor': 'Dr. Emily Johnson', 'subject': 'Physics', 'stars': 5, 'review': "Dr. Johnson's lectures are incredibly engaging and she explains complex concepts with great clarity. Her passion for physics is contagious!"}


TypeError: embed_content() got an unexpected keyword argument 'text'